In [369]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from sklearn.svm import LinearSVC

In [326]:
train_data = pd.read_csv('data/processed_data.csv')
train_data.dropna(inplace=True)
train_data.head()

,tweet_id,tweet,sentiment
0,1701,sxswnui sxsw apple defining language touch dif...,1
1,1851,learning google doodles doodles light funny in...,1
2,2689,one -your-face stealing show years sxsw apple ...,2
3,4525,iphone sxsw app would pretty awesome crash eve...,0
4,3604,line outside apple store austin waiting new ip...,1


In [327]:
train_withstop = pd.read_csv('data/train_withstop.csv')
train_withstop.dropna(inplace=True)
train_withstop.head()

,tweet_id,tweet,sentiment
0,1701,sxswnui sxsw apple defining language touch wit...,1
1,1851,learning about google doodles all doodles shou...,1
2,2689,one the most -your-face stealing the show year...,2
3,4525,this iphone sxsw app would pretty awesome didn...,0
4,3604,line outside the apple store austin waiting fo...,1


In [328]:
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [329]:
train_data['sentiment'].value_counts()

1    4309
2    2382
0     456
3     125
Name: sentiment, dtype: int64

In [330]:
# tfidf_vectorizer = TfidfVectorizer()
# X = tfidf_vectorizer.fit_transform(train_data['tweet'])

In [331]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
X = count_vectorizer.fit_transform(train_data['tweet'])

In [332]:
X.shape

(7272, 80022)

In [370]:
ros = RandomOverSampler(random_state=3)
data_res, label_res = ros.fit_sample(X, train_data['sentiment'])

In [371]:
# smt = SMOTE(random_state=777, k_neighbors=1)
# data_res, label_res = smt.fit_sample(X, train_data['sentiment'])

In [372]:
data_res.shape

(17236, 80022)

In [368]:
label_res.shape

(500,)

In [355]:
# model = pipeline_sgd.fit(data_res, label_res)

In [356]:
test_data = pd.read_csv('data/test_processed.csv')
test_data.head()

,tweet_id,tweet
0,7506,audience prototyping tools use sketchbooks/sha...
1,7992,sxsw send best photos videos .. link citizen_j...
2,247,pic winning ipad unsix sxsw link cont link
3,7688,google marissa mayer mobile phone cursor physi...
4,3294,sxsw google maps even cooler thought


In [357]:
# svc_pred = model.predict(test_withstop['tweet'])

In [358]:
X_test = count_vectorizer.transform(test_data['tweet'])

In [359]:
X_test.shape

(1819, 80022)

In [360]:
#fit and predict on upsampled data
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=7600)
lr_model.fit(data_res, label_res)
lr_pred = lr_model.predict(X_test)

In [361]:
# svc = LinearSVC(penalty='l2', C=0.5, dual=False, random_state=0, max_iter=1000)
# svc.fit(X, train_data['sentiment'])
# svc_pred = svc.predict(X_test)
# svc_pred

In [363]:
to_submit = pd.DataFrame({'tweet_id': test_data['tweet_id'], 'sentiment': lr_pred})
to_submit.to_csv('solutions/lr_undersample.csv', index=False)
to_submit.head()

,tweet_id,sentiment
0,7506,2
1,7992,2
2,247,2
3,7688,2
4,3294,2


In [254]:
to_submit = pd.DataFrame({'tweet_id': test_data['tweet_id'], 'sentiment': svc_pred})
to_submit.to_csv('solutions/svc_upsampled_lemmatized.csv', index=False)
to_submit.head()

,tweet_id,sentiment
0,7506,1
1,7992,1
2,247,1
3,7688,2
4,3294,1


In [27]:
to_submit = pd.DataFrame({'tweet_id': test_withstop['tweet_id'], 'sentiment': svc_pred})
to_submit.to_csv('solutions/svc_pred_withstop.csv', index=False)
to_submit.head()

,tweet_id,sentiment
0,7506,1
1,7992,1
2,247,1
3,7688,2
4,3294,2
